In [1]:
from random import uniform
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import random

## Configs

In [2]:
POPULATION = 10
GOOD_PARENTS_CNT = 3
LUCKY_PARENTS_CNT = 1
CHILDREN_CNT = 5
MUTATION_PROB = 0.1

## Load the image and flatten it

In [3]:
# Load Image
jpg_img_arr = mpimg.imread('Bigger-Splash-1967.jpg')
jpg_IMG = Image.open('Bigger-Splash-1967.jpg')
FLATTENED = jpg_img_arr.flatten()
LEN_FLATTENED = len(FLATTENED)
print("flattend size :", FLATTENED.shape)

flattend size : (1769472,)


## Initial Population

In [5]:
# Initial population
np.random.seed(0)

## POPULATION만큼 초기값 생성
initial_population = np.array([np.random.randint(255) for _ in range(POPULATION)])
print(initial_population.shape)

(100,)


In [55]:
class Generation:
    cnt = 0
    def __init__(self, population):
        Generation.cnt += 1
        self.generation_lv = Generation.cnt
        self.population = population
        self.sorted_pop = self.sort_pop()
    
    def __repr__(self):
        return "Generation Level : {}".format(self.generation_lv)
    
 
    def evolution(self):
        print("Start Evolution Generation level %d" % Generation.cnt)
        # next_generation = list()
        # select Good_parents
        good_parents = self.sorted_pop[:GOOD_PARENTS_CNT]
        
        # select Lucky_parents
        lucky_parents = random.sample(self.sorted_pop[GOOD_PARENTS_CNT:], LUCKY_PARENTS_CNT)
        
        print("good :", good_parents)
        print("lucky :", lucky_parents)
        
        # make Childeren
        good_child = self.make_children(good_parents)
        lucky_child = self.make_children(lucky_parents)
        
        # merge Childeren
        children = good_child + lucky_child
        random.shuffle(children)
            
        # make mutations
        children = self.make_mutation(children)
            
        return children
    
    def sort_pop(self):
        sorted_pop = sorted(self.population, key=lambda x: x.fitness, reverse=False)
        
        return sorted_pop
    
    def make_children(self, parents):
        children = list()
        
        # crossover
        for i in range(int(len(parents) / 2)):
            # 한개는 중간에서 반반 섞기
            split_child = parents[i].chromo[:int(LEN_FLATTENED/2)] + parents[len(parents)-1-i].chromo[:int(LEN_FLATTENED/2)]
            children.append(split_child)
            
            # 나머지는 랜덤으로 가져오기
            for _ in range(CHILDREN_CNT - 1):
                random_child = random.sample(parents[i].chromo + parents[len(parents)-1-i].chromo, LEN_FLATTENED)
                children.append(random_child)
        
        return children
    
    def make_mutation(self, children):
        # mutated = list()
        mutation_cnt = int(len(children) * MUTATION_PROB)
        for i in range(mutation_cnt):
            children[i] = [np.random.randint(255) for _ in range(LEN_FLATTENED)]
        
        return children
    
    def fitness(self):
        return np.mean([chromo.fitness for chromo in self.population])

In [56]:
dna = [chromoSome() for _ in range(POPULATION)]
a = Generation(dna)

In [ ]:
aa = a.evolution()

Start Evolution Generation level 1
good : [chromosome fitenss : 154069546, chromosome fitenss : 154089234, chromosome fitenss : 154110022]
lucky : [chromosome fitenss : 154324860]


In [5]:
class chromoSome:
    def __init__(self, chromo_data=None):
        if chromo_data is None:
            self.chromo = [np.random.randint(255) for _ in range(len(FLATTENED))]
        else:
            self.chromo = chromo_data
    
    def __repr__(self):
        return "chromosome fitenss : {}".format(self.fitness)
    
    @property
    def fitness(self):
        # score가 0에 가까우면 좋은 것
        score = 0
        for gene, wanted in zip(self.chromo, FLATTENED):
            score += abs(gene - wanted)
        
        return score
            
    

In [ ]:
# Fitness function
# Selection
# Crossover
# Mutation

In [18]:
w, h = 512, 512
data = np.zeros((h, w, 3), dtype=np.uint8)
data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
img = Image.fromarray(data, 'RGB')
img.show()